# This notebook: Build evaluation method
* Aim at >.90 accuracy

currently it is tested with yolov5 prediction results.

It is compatible for all torch prediction outputs in the form of .pandas().xywh (xcenter, ycenter, width, height).

Using Google Colab to view this notebook is highly recommended.


In [ ]:
!nvidia-smi

Fri Oct 14 16:04:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    45W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 0. Prep works, install yolov5, download and partition datasets

In [ ]:
%%capture
!git clone https://github.com/ultralytics/yolov5 
%cd yolov5
!pip install -r requirements.txt #wandb
%cd ..

In [ ]:
from PIL import Image, ExifTags
from pycocotools.coco import COCO
from matplotlib.patches import Polygon, Rectangle
from matplotlib.collections import PatchCollection
import colorsys
import random
import pylab

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from tqdm import tqdm

import shutil
import os
import re


import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [ ]:
mount_drive = False
if mount_drive:
  # gdown a gdrive file too frequently triggers google's control and makes the file un-gdown-able
  # in this case, go to 1hq0KcSM31yrR4YlWqM_P29Y3YTuvuIom and 1X3O2v3GIPveq3ylWF6o1qHI5uzbN1vWA, manually
  # make a copy of them to your own drive and mount your drive to the colab instance, then you can manipulate freely
  from google.colab import drive
  drive.mount('/content/drive')
  %cp ./drive/MyDrive/rotated2.zip ./
  %cp /content/drive/MyDrive/trash_ai_trained_weights/yolov5x6_best_weights.pt ./
  else:
    !gdown 1hq0KcSM31yrR4YlWqM_P29Y3YTuvuIom # download best trained yolov5x6 weights
    !gdown 1X3O2v3GIPveq3ylWF6o1qHI5uzbN1vWA # download organized TACO images (TACO itself, 1500 images, without unofficial images)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -qq ./rotated2.zip 
%mv ./content/* ./

In [ ]:
%%capture
!wget https://raw.githubusercontent.com/pedropro/TACO/master/data/annotations.json
!wget https://raw.githubusercontent.com/pedropro/TACO/master/data/annotations_unofficial.json

In [ ]:
nr_imgs=None
for root, dirnames, filenames in os.walk('./yoloTACO/labels/'):
  nr_imgs = len(filenames)
  break
print('Number of all images:\n'+str(nr_imgs))

## train test split
'''
train: images/train
val: images/val
test: images/test
'''
np.random.seed(5)
id_list=[i for i in range(nr_imgs)]
np.random.shuffle(id_list)
train_ids = id_list[:1300]
val_ids = id_list[1300:1400]
test_ids = id_list[1400:]

def move_helper(ids, desti):
  for id in ids:
    img_name = os.path.join( './yoloTACO/images', str(id)+'.jpg' )
    lbl_name = os.path.join( './yoloTACO/labels', str(id)+'.txt' )
    print(img_name)
    if os.path.isfile(img_name):
        shutil.copy( img_name, './yoloTACO/images/'+desti)
        shutil.copy( lbl_name, './yoloTACO/labels/'+desti)
    else :
        print('file does not exist', img_name)

Number of all images:
1500


In [ ]:
%%capture
!mkdir yoloTACO/images/train
!mkdir yoloTACO/images/val
!mkdir yoloTACO/images/test
!mkdir yoloTACO/labels/train
!mkdir yoloTACO/labels/val
!mkdir yoloTACO/labels/test
move_helper(test_ids,'test')
move_helper(train_ids,'train')
move_helper(val_ids,'val')

In [ ]:
%%bash
mkdir ./datasets
mv yoloTACO datasets/

In [ ]:
#@title yml

with open('./yolov5/data/yoloTACO.yaml', mode='w') as fp:
  lines = '''path: ../datasets/yoloTACO  # dataset root dir
train: images/train  # train images 
val: images/val  # val images 
test: images/test # test images (optional)

# Classes
names:
  0: Aluminium foil
  1: Battery
  2: Aluminium blister pack
  3: Carded blister pack
  4: Other plastic bottle
  5: Clear plastic bottle
  6: Glass bottle
  7: Plastic bottle cap
  8: Metal bottle cap
  9: Broken glass
  10: Food Can
  11: Aerosol
  12: Drink can
  13: Toilet tube
  14: Other carton
  15: Egg carton
  16: Drink carton
  17: Corrugated carton
  18: Meal carton
  19: Pizza box
  20: Paper cup
  21: Disposable plastic cup
  22: Foam cup
  23: Glass cup
  24: Other plastic cup
  25: Food waste
  26: Glass jar
  27: Plastic lid
  28: Metal lid
  29: Other plastic
  30: Magazine paper
  31: Tissues
  32: Wrapping paper
  33: Normal paper
  34: Paper bag
  35: Plastified paper bag
  36: Plastic film
  37: Six pack rings
  38: Garbage bag
  39: Other plastic wrapper
  40: Single-use carrier bag
  41: Polypropylene bag
  42: Crisp packet
  43: Spread tub
  44: Tupperware
  45: Disposable food container
  46: Foam food container
  47: Other plastic container
  48: Plastic glooves
  49: Plastic utensils
  50: Pop tab
  51: Rope & strings
  52: Scrap metal
  53: Shoe
  54: Squeezable tube
  55: Plastic straw
  56: Paper straw
  57: Styrofoam piece
  58: Unlabeled litter
  59: Cigarette'''
  fp.writelines(lines)

In [ ]:
%cd ./yolov5
!ls

/content/yolov5
benchmarks.py	 detect.py   models	       setup.cfg       val.py
classify	 export.py   README.md	       train.py
CONTRIBUTING.md  hubconf.py  requirements.txt  tutorial.ipynb
data		 LICENSE     segment	       utils


In [ ]:
%pwd

'/content/yolov5'

# 1. Evaluate with our best trained weights so far

## detect and eval with yolo default scripts

In [ ]:
!python val.py --data yoloTACO.yaml --task test --weights ./yolov5x6_best_weights.pt
#!python detect.py --weights ./yolov5x6_best_weights.pt --source /content/datasets/yoloTACO/images/test

val: data=/content/yolov5/data/yoloTACO.yaml, weights=['./yolov5x6_best_weights.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.2-194-g2a19d07 Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 416 layers, 140537980 parameters, 0 gradients, 209.1 GFLOPs
test: Scanning '/content/datasets/yoloTACO/labels/test' images and labels...100 found, 0 missing, 0 empty, 0 corrupt: 100% 100/100 [00:00<00:00, 2635.51it/s]
test: New cache created: /content/datasets/yoloTACO/labels/test.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 4/4 [00:08<00:00,  2.16s/it]
                   all        100        300     0.0464      0.628       0.11      0.102

Note that the default `MAP` is not the "wanted" metrics for our project, as our sponsor specifically requested a metrics under the name "accuracy" and a target score of >.90.

## detect with torch framework manually

This is a necessary step to use our accuracy evaluator.

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./yolov5x6_best_weights.pt')  # load our local model

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-10-14 Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 416 layers, 140537980 parameters, 0 gradients, 209.1 GFLOPs
Adding AutoShape... 


In [ ]:
# Load test imgs
test_dir = '/content/datasets/yoloTACO/images/test/'
test_list = [i[2] for i in os.walk(test_dir)][0]
test_list = [re.findall(r'\d+',i)[0] for i in test_list]
test_read_img_list = [Image.open(test_dir+str(i)+'.jpg') for i in test_list]
# alternatively use cv2: cv2.imread('target_path')[..., ::-1]  # OpenCV image (BGR to RGB)


In [ ]:
# Inference
results = model(test_read_img_list) # batch of images
pred_pd = results.pandas().xywh 

In [ ]:
%%capture
!wget -O data/annotations.json https://raw.githubusercontent.com/pedropro/TACO/master/data/annotations.json
anno_path = './data/annotations.json'
annos = COCO(annotation_file=anno_path)
with open(anno_path, 'r') as f:
    annos_json = json.loads(f.read())
no_to_clname = {i:j for i,j in enumerate([i['name'] for i in annos_json['categories']])}


In [ ]:
truth = [annos.loadAnns(annos.getAnnIds(int(i))) for i in test_list]
truth_pd = []
for i in truth:
  cache = [j['bbox']+[1]+[j['category_id']]+[no_to_clname[j['category_id']]] for j in i]
  df = pd.DataFrame(cache,columns = ['xcenter','ycenter','width','height','confidence','class','name'])
  df['xcenter'] = df['xcenter'] + df['width']/2
  df['ycenter'] = df['ycenter'] + df['height']/2
  truth_pd.append(df)

# 2. Accuracy evaluation

For each object with a truth bounding box in each image, if there is a prediction bounding box that has an IOU > threshold with that truth bounding box, it is counted as `detected`.

For overall model `accuracy`, we count total number of `detected` of all images over total number of `objects` of all images.

In [ ]:
def bbox_iou(box1, box2, eps=1e-7):
  # CITATION: adapted from YOLOV5 utils, author, cr: ultralytics
  # Returns Intersection over Union (IoU) of box1(1,4) to box2(n,4)

  # Get the coordinates of bounding boxes, transform from xywh to xyxy
  (x1, y1, w1, h1), (x2, y2, w2, h2) = box1.chunk(4, 1), box2.chunk(4, 1)
  w1_, h1_, w2_, h2_ = w1 / 2, h1 / 2, w2 / 2, h2 / 2
  b1_x1, b1_x2, b1_y1, b1_y2 = x1 - w1_, x1 + w1_, y1 - h1_, y1 + h1_
  b2_x1, b2_x2, b2_y1, b2_y2 = x2 - w2_, x2 + w2_, y2 - h2_, y2 + h2_


  inter = (torch.min(b1_x2, b2_x2) - torch.max(b1_x1, b2_x1)).clamp(0) * \
          (torch.min(b1_y2, b2_y2) - torch.max(b1_y1, b2_y1)).clamp(0)
  union = w1 * h1 + w2 * h2 - inter + eps
  return inter / union  # return IoU
  
def each_pic(pred_df,truth_df,iou_th):
  nr_objs = truth_df.shape[0]
  nr_dets = 0
  for i in truth_df.iterrows():
    tbox_tensor = torch.tensor([i[1].tolist()[:4]])
    tlabel = i[1].tolist()[5]
    
    row_counter=0
    for j in pred_df.iterrows():
      pbox_tensor = torch.tensor([j[1].tolist()[:4]])
      plabel = j[1].tolist()[5]
      if bbox_iou(tbox_tensor,pbox_tensor)>iou_th and tlabel==plabel:
        nr_dets+=1
        pred_df.drop([row_counter]) # drop matched bbox, so one prediction bbox 
                                    # wont be counted as "detected" for two different objects
        continue
      row_counter+=1
  return nr_objs,nr_dets

In [ ]:
def acc(pred,truth,iou_th=0.7):
  objs,dets=0,0
  for i in tqdm(range(len(truth))):
    o,d=each_pic(pred_pd[i],truth_pd[i],iou_th)
    objs+=o
    dets+=d
  return np.round(dets/objs,6)

accuracy = acc(pred_pd,truth_pd)

100%|██████████| 100/100 [00:00<00:00, 190.89it/s]


In [ ]:
print('Our trained model has an accuracy of: '+str(accuracy))

Our trained model has an accuracy of: 0.6
